### Housekeeping - Setup & wipe out any prior records on the Aerospike Server

We have a namespace **_test_** pre-defined on the server. Lets truncate it using aql.

This is needed while doing code development. If you want to clear the iJava Kernel of all Java objects and run all cells from scratch, Kernel->Restart & Run All, this will ensure any records written on the underlying Aerospike cluster are purged.

First, we need required imports for using %sh in interactive Java Kernel. (This is specific to the iJava Kernel implementation by **Spencer Park** that we are using.)

In [9]:
import io.github.spencerpark.ijava.IJava;
import io.github.spencerpark.jupyter.kernel.magic.common.Shell;
IJava.getKernelInstance().getMagics().registerMagics(Shell.class);

### Runing aql in iJava
We can run aql commands in a file. No output will be displayed. 
In _trucate_test.aql_, we have a single AQL command: TRUNCATE test



In [10]:
%sh aql -c "truncate test" -h "localhost" -p 3000

#### Add Java Client POM Dependency  
Jupyter Notebook way!

In [11]:
%%loadFromPOM
<dependencies>
  <dependency>
    <groupId>com.aerospike</groupId>
    <artifactId>aerospike-client</artifactId>
    <version>6.0.0</version>
  </dependency>
</dependencies>

####  Java Client Imports and Test code

In [12]:
//Required Imports
import com.aerospike.client.AerospikeClient;
import com.aerospike.client.policy.WritePolicy;
import com.aerospike.client.Bin;
import com.aerospike.client.Key;
import com.aerospike.client.Record;
import com.aerospike.client.Value;

//import com.aerospike.client.cdt.ListPolicy;
//import com.aerospike.client.cdt.ListOperation;
//import com.aerospike.client.cdt.ListOrder;
//import com.aerospike.client.cdt.ListWriteFlags;
//import com.aerospike.client.cdt.ListReturnType;

//import com.aerospike.client.cdt.MapPolicy;
//import com.aerospike.client.cdt.MapOrder;
//import com.aerospike.client.cdt.MapWriteFlags;
//import com.aerospike.client.cdt.MapReturnType;

//import com.aerospike.client.exp.MapExp;
//import com.aerospike.client.cdt.CTX;
//import java.util.HashMap;
//import java.util.Map;

import com.aerospike.client.Operation;
import com.aerospike.client.query.Statement;
import com.aerospike.client.exp.Exp;
//import com.aerospike.client.cdt.MapOperation;

//import com.aerospike.client.policy.BatchPolicy;

//import java.util.Random;f





In [15]:
AerospikeClient client = new AerospikeClient("localhost", 3000);

//AerospikeClient client = new AerospikeClient("localhost", 3000);
System.out.println("Initialized the client and connected to the cluster.");


Initialized the client and connected to the cluster.


In [41]:
//Insert a record using a class based construct
import com.aerospike.client.exp.ExpOperation;
import com.aerospike.client.exp.*;


String ns = "test";
String setName = "testset";
int profileId = 2;
String name = "me";
int zip = 94506;
    
Key user = new Key(ns, setName, profileId); 
    
Bin bProfileId = new Bin("ProfileId", profileId);
Bin bName = new Bin("Name", name);
Bin bZip = new Bin("Zip", zip);
    
WritePolicy wPolicy = new WritePolicy();
wPolicy.durableDelete = false;
//wPolicy.expiration = -1;
      

Record rec = client.operate(wPolicy, user, 
                ExpOperation.read("retval", Exp.build(Exp.deviceSize()), ExpReadFlags.DEFAULT),
                Operation.put(bProfileId),
                Operation.put(bName),
                Operation.put(bZip)
                //ExpOperation.write("ProfileId", Exp.build(Exp.val(0)), ExpWriteFlags.DEFAULT),
                //ExpOperation.read("retvalafter", Exp.build(Exp.deviceSize()), ExpReadFlags.DEFAULT)    
             ) ; 

System.out.println(rec.getValue("retval"));
//System.out.println(rec.getValue("retvalafter"));


//client.put(wPolicy1, user, bProfileId, bName, bZip);
     
//List<?> retList = (ArrayList<?>)rec.getList();
//System.out.println("\nExists"+ retList.get(0));
//System.out.println("\nPut"+ retList.get(1));
    
System.out.println(client.get(null, user));
    

16
(gen:1),(exp:431243991),(bins:(ProfileId:2),(Name:me),(Zip:94506))


In [40]:
client.delete(wPolicy, user);

true

In [36]:
client.get(null,user);

(gen:4),(exp:431243964),(bins:(ProfileId:2),(Name:me),(Zip:94506))

In [37]:
%sh aql -c "truncate test" -h "localhost" -p 3000

In [ ]:
userProfile pf = new userProfile();
pf.createProfile("test", "testset", 0,"Test", 94506, true);

In [ ]:
userProfile pf = new userProfile();
pf.createProfile("test", "testset", 4,"Test", 94506, false);

In [ ]:
Key user4 = new Key("test", "testset", 4); 
System.out.println(client.get(null, user4));

In [ ]:
class Truncate {
  public void truncate(String ns, String setName, int sinceUpdate_ms) {
    WritePolicy wPolicy = new WritePolicy();
    if(sinceUpdate_ms > 0){
      //Add "Since Last Update Time" based Expression Filter
      wPolicy.filterExp = Exp.build(Exp.gt(Exp.sinceUpdate(), Exp.val(sinceUpdate_ms)));
    }
    
    Operation[] ops = new Operation[] {
      Operation.delete()
    };
    Statement stmt = new Statement();
    stmt.setNamespace(ns);
    if(setName != null){
      stmt.setSetName(setName);
    }
    stmt.setOperations(ops);
    client.execute(wPolicy, stmt);
  }
}

In [ ]:
Truncate tr = new Truncate();
tr.truncate("test", "testset", 0);

In [ ]:
System.out.println(client.get(null, user4));

In [ ]:
pf.createProfile("test", "testset1", 1,"Test", 94506, false);
pf.createProfile("test", "testset2", 2,"Test", 94506, false);
pf.createProfile("test", "testset3", 3,"Test", 94506, false);


In [ ]:
Key user1 = new Key("test", "testset1", 1); 
Key user2 = new Key("test", "testset2", 2); 
Key user3 = new Key("test", "testset3", 3);

System.out.println(client.get(null, user1));
System.out.println(client.get(null, user2));
System.out.println(client.get(null, user3));

In [ ]:
Truncate tr = new Truncate();
tr.truncate("test", null, 10000);   
//Re-run afer 10 seconds to test ... should truncate only 10 seconds after creation.

In [ ]:
System.out.println(client.get(null, user1));
System.out.println(client.get(null, user2));
System.out.println(client.get(null, user3));